In [ ]:
# global B2, h_opt, J


def indicator_matrix(row):
    tmp = row.sigma.copy()
    tmp[row.idx] = 0
    return np.diag(tmp)

def compute_Luk(row, b2, J):
    Lu = b2 @ row.sigma @ b2.T
    Luk = np.array([la.matrix_power(Lu, i) for i in range(1, J + 1)])
    return Luk

def greedy_search(Y_test: np.ndarray, 
                    J: int, 
                    P: int,
                    m_test:int,
                    Ldk: np.ndarray,
                    h_opt: np.ndarray, 
                    K0: int,
                    B2: np.ndarray,
                    filter: np.ndarray = 1):

    search_space = np.where(filter == 1)    
    sigmas = pd.DataFrame({"idx": search_space[0]})

    sigmas["sigma"] = sigmas.idx.apply(lambda x: filter)
    sigmas["sigma"] = sigmas.apply(lambda x: indicator_matrix(x), axis=1)
    sigmas["Luk"] = sigmas.apply(lambda x: compute_Luk(x, B2, J), axis=1)
    sigmas["D"] = sigmas.apply(lambda x: generate_dictionary(h_opt, P, x.Luk, Ldk), axis=1)
    sigmas["X"] = sigmas.D.apply(lambda x: sparse_transform(x, K0, Y_test))
    sigmas["NMSE"] = sigmas.apply(lambda x: nmse(x.D, x.X, Y_test, m_test), axis=1)

    return sigmas


def learn_upper_laplacian(Y_train: np.ndarray,
                          Y_test: np.ndarray, 
                          J: int, 
                          M: int, 
                          P: int,
                          m_test: int,
                          Lu: np.ndarray, 
                          Ld: np.ndarray,
                          dictionary_type: str, 
                          c: float, 
                          epsilon: float, 
                          K0: int,
                          B2: np.ndarray,
                          filter: np.ndarray = 1,
                          current_min: float = None,
                          lambda_: float = 1e-3, 
                          max_iter: int = 10, 
                          patience: int = 10,
                          tol: float = 1e-7, 
                          verbose: bool = False):
    
    
    Ldk,_,_= compute_Lj_and_lambdaj(Ld, J)

    D0, X0, _ = initialize_dic(Lu=Lu,
                               Ld=Ld,
                               P=P,
                               J=J,
                               Y_train=Y_train,
                               K0=K0,
                               dictionary_type=dictionary_type,
                               c=c,
                               epsilon=epsilon,
                               only="only_X")

    _, min_error_test, h_opt, X_opt_test, _, D_opt, Ldk, history = topological_dictionary_learn(Y_train=Y_train,
                                                                                    Y_test=Y_test,
                                                                                    J=J,
                                                                                    M=M,
                                                                                    P=P,
                                                                                    D0=D0,
                                                                                    X0=X0,
                                                                                    Lu=Lu,
                                                                                    Ld=Ld,
                                                                                    dictionary_type=dictionary_type,
                                                                                    c=c,
                                                                                    epsilon=epsilon,
                                                                                    K0=K0,
                                                                                    lambda_=lambda_,
                                                                                    max_iter=max_iter,
                                                                                    patience=patience,
                                                                                    tol=tol)
    tot_hist = []
    Lu_opt = Lu
    current_min = -np.inf
    T = B2.shape[1]
    filter = np.ones(T)

    while current_min<min_error_test:
        
        sigmas = greedy_search(Y_test,
                                J,
                                P,
                                m_test,
                                Ldk,
                                h_opt,
                                K0,
                                B2,
                                filter)
    
        current_min = sigmas.NMSE.min()
        idx_min = sigmas.NMSE.idxmin()
        print(current_min)
        if current_min < min_error_test:
            S = sigmas.sigma[idx_min]
            Lu_opt = B2 @ S @ B2.T
            filter = np.diagonal(S)

            D0, X0, _ = initialize_dic(Lu=Lu,
                            Ld=Ld,
                            P=P,
                            J=J,
                            Y_train=Y_train,
                            K0=K0,
                            dictionary_type=dictionary_type,
                            c=c,
                            epsilon=epsilon,
                            only="only_X")

            _, min_error_test, h_opt, X_opt_test, _, D_opt, Ldk, history = topological_dictionary_learn(Y_train=Y_train,
                                                                                            Y_test=Y_test,
                                                                                            J=J,
                                                                                            M=M,
                                                                                            P=P,
                                                                                            D0=D0,
                                                                                            X0=X0,
                                                                                            Lu=Lu_opt,
                                                                                            Ld=Ld,
                                                                                            dictionary_type=dictionary_type,
                                                                                            c=c,
                                                                                            epsilon=epsilon,
                                                                                            K0=K0,
                                                                                            lambda_=lambda_,
                                                                                            max_iter=max_iter,
                                                                                            patience=patience,
                                                                                            tol=tol)

            tot_hist.append(history)

            if verbose:
                print(f'Removing 1 triangle from topology... \n ... New min test error: {current_min} !')

    return min_error_test, tot_hist, Lu_opt, h_opt, X_opt_test, D_opt


In [ ]:
import warnings

warnings.filterwarnings('ignore')

min_error_test, tot_hist, Lu_opt, h_opt, X_opt_test, D_opt = learn_upper_laplacian(B2=B2,
                                                        Y_train=Y_train[:,:,s],
                                                        Y_test=Y_test[:,:,s],
                                                        J=J,
                                                        M=M,
                                                        P=P,
                                                        m_test=m_test,
                                                        Lu=Lu_full,
                                                        Ld=Ld,
                                                        dictionary_type=dictionary_type,
                                                        c=c,
                                                        epsilon=epsilon,
                                                        K0=k0,
                                                        lambda_=lambda_,
                                                        max_iter=max_iter,
                                                        patience=patience,
                                                        tol=tol,
                                                        verbose=verbose)